In [12]:
import os
import json
import jax
from time import time
from jax import config
import numpy as np
import jax.numpy as jnp
import flax.linen as nn
from termcolor import colored

from src.utils import *
from src.boundary_conditions import *
from src.models import BGKSim, KBCSim
from src.lattice import LatticeD2Q9

# Use 8 CPU devices
# os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8'
jax.config.update('jax_enable_x64', True)


In [2]:
class Cylinder(BGKSim):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def set_boundary_conditions(self):
        # Define the cylinder surface
        coord = np.array([(i, j) for i in range(self.nx) for j in range(self.ny)])
        xx, yy = coord[:, 0], coord[:, 1]
        cx, cy = 2.*_diam, 2.*_diam
        cylinder = (xx - cx)**2 + (yy-cy)**2 <= (_diam/2.)**2
        cylinder = coord[cylinder]
        implicit_distance = np.reshape((xx - cx)**2 + (yy-cy)**2 - (_diam/2.)**2, (self.nx, self.ny))
        self.BCs.append(InterpolatedBounceBackBouzidi(tuple(cylinder.T), implicit_distance, self.gridInfo, self.precisionPolicy))

        # Outflow BC
        outlet = self.boundingBoxIndices['right']
        rho_outlet = np.ones((outlet.shape[0], 1), dtype=self.precisionPolicy.compute_dtype)
        self.BCs.append(ExtrapolationOutflow(tuple(outlet.T), self.gridInfo, self.precisionPolicy))
        # self.BCs.append(ZouHe(tuple(outlet.T), self.gridInfo, self.precisionPolicy, 'pressure', rho_outlet))

        # Inlet BC
        inlet = self.boundingBoxIndices['left']
        rho_inlet = np.ones((inlet.shape[0], 1), dtype=self.precisionPolicy.compute_dtype)
        vel_inlet = np.zeros(inlet.shape, dtype=self.precisionPolicy.compute_dtype)
        yy_inlet = yy.reshape(self.nx, self.ny)[tuple(inlet.T)]
        vel_inlet[:, 0] = poiseuille_profile(yy_inlet,
                                             yy_inlet.min(),
                                             yy_inlet.max()-yy_inlet.min(), 3.0 / 2.0 * _prescribed_vel)
        self.BCs.append(Regularized(tuple(inlet.T), self.gridInfo, self.precisionPolicy, 'velocity', vel_inlet))

        # No-slip BC for top and bottom
        wall = np.concatenate([self.boundingBoxIndices['top'], self.boundingBoxIndices['bottom']])
        vel_wall = np.zeros(wall.shape, dtype=self.precisionPolicy.compute_dtype)
        self.BCs.append(Regularized(tuple(wall.T), self.gridInfo, self.precisionPolicy, 'velocity', vel_wall))

    def output_data(self, **kwargs):
        self.saved_data.append(kwargs)
        # 1:-1 to remove boundary voxels (not needed for visualization when using bounce-back)
    
    def get_force(self):
        pass

# Helper function to specify a parabolic poiseuille profile
poiseuille_profile  = lambda x,x0,d,umax: np.maximum(0.,4.*umax/(d**2)*((x-x0)*d-(x-x0)**2))

In [140]:
from flax import linen as nn

class ConvBlock(nn.Module):
  """Defines a convolutional block with activation and normalization."""
  features: int
  kernel_size: int = (3,3)
  strides: int = 1

  @nn.compact
  def __call__(self, inputs):
    x = nn.Conv(self.features, kernel_size=self.kernel_size, strides=self.strides, padding='SAME')(inputs)
    x = nn.BatchNorm(use_running_average=True)(x)
    x = nn.relu(x)
    return x

class DownBlock(nn.Module):
  """Downsamples feature maps through convolutions and pooling."""
  features: int
  pool_factor: int = 2

  @nn.compact
  def __call__(self, x):
    x = ConvBlock(self.features)(x)
    x = ConvBlock(self.features)(x)
    return x

class UpBlock(nn.Module):
  """Upsamples feature maps and concatenates with features from the contracting path."""
  features: int
  up_factor: int = 2

  @nn.compact
  def __call__(self, x):
    x = ConvBlock(self.features)(x)
    x = ConvBlock(self.features)(x)
    x = nn.ConvTranspose(self.features, kernel_size=(2, 2), strides=self.up_factor, padding='VALID')(x)
    return x

class UNet(nn.Module):
  """UNet architecture with contracting and expanding paths."""
  features_start: int = 64

  @nn.compact
  def __call__(self, x):
    input_shape = x.shape
    # Contracting path
    down1 = DownBlock(self.features_start * 2)(x)
    down1_max_pooled = nn.max_pool(down1, window_shape=(2, 2), strides=(2, 2))
    down2 = DownBlock(self.features_start * 4)(down1_max_pooled)
    down2_max_pooled = nn.max_pool(down2, window_shape=(2, 2), strides=(2, 2))
    down3 = DownBlock(self.features_start * 8)(down2_max_pooled)
    down3_max_pooled = nn.max_pool(down3, window_shape=(2, 2), strides=(2, 2))
    down4 = DownBlock(self.features_start * 16)(down3_max_pooled)
    down4_max_pooled = nn.max_pool(down4, window_shape=(2, 2), strides=(2, 2))
    
    # Expanding path with concatenation
    up1 = UpBlock(self.features_start * 16)(down4_max_pooled)
    down4_sliced = jax.lax.slice(down4, (4, 4, 0),(down4.shape[0]-4, down4.shape[1]-4, down4.shape[2]))
    up1_concatenated = jax.lax.concatenate([down4_sliced, up1], dimension=2)
    up2 = UpBlock(self.features_start * 4)(up1_concatenated)
    down3_sliced = jax.lax.slice(down3, (4, 4, 0), (down3.shape[0]-4, down3.shape[1]-4, down3.shape[2]))
    up2_concatenated = jax.lax.concatenate([down3_sliced, up2], dimension=2)
    up3 = UpBlock(self.features_start * 2)(up2_concatenated)
    print(up3.shape)
    return up3
  
class SimpleNet(nn.Module):
    features: int = 32
    kernel_size: int = (5, 5)
    strides: int = 1
    @nn.compact
    def __call__(self, x):
        x = nn.Conv(self.features, kernel_size=self.kernel_size, strides=self.strides, padding='SAME')(x)
        x = nn.leaky_relu(x)
        x = nn.Conv(self.features, kernel_size=self.kernel_size, strides=self.strides, padding='SAME')(x)
        x = nn.leaky_relu(x)
        x = nn.Conv(self.features, kernel_size=self.kernel_size, strides=self.strides, padding='SAME')(x)
        x = nn.leaky_relu(x)
        x = nn.Conv(2, kernel_size=self.kernel_size, strides=self.strides, padding='SAME')(x)
        return x
        

In [142]:
my_unet = SimpleNet()
print(my_unet.tabulate(jax.random.key(0), jnp.ones((64,32,3)),
                   compute_flops=True, compute_vjp_flops=True)) # check parameters, should be 55,298 in total


                               SimpleNet Summary                                
┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ path   ┃ module    ┃ inputs     ┃ outputs   ┃ flops ┃ vjp_flops ┃ params     ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│        │ SimpleNet │ float64[6… │ float64[… │ 0     │ 0         │            │
├────────┼───────────┼────────────┼───────────┼───────┼───────────┼────────────┤
│ Conv_0 │ Conv      │ float64[6… │ float64[… │ 0     │ 0         │ bias:      │
│        │           │            │           │       │           │ float32[3… │
│        │           │            │           │       │           │ kernel:    │
│        │           │            │           │       │           │ float32[5… │
│        │           │            │           │       │           │            │
│        │           │            │           │       │           │ 2,432 (9.7 │
│        │           │     

In [3]:
def generate_sim_dataaset(diam, timestep_offset, timestep_stride, timestep_end):
    global _diam
    global _prescribed_vel
    _diam = diam
    precision = 'f64/f64'
    # diam_list = [10, 20, 30, 40, 60, 80]
    scale_factor = 80 / diam
    prescribed_vel = 0.003 * scale_factor
    _prescribed_vel = prescribed_vel
    lattice = LatticeD2Q9(precision)

    nx = int(22*diam)
    ny = int(4.1*diam)

    Re = 100.0
    visc = prescribed_vel * diam / Re
    omega = 1.0 / (3. * visc + 0.5)
    
    os.system('rm -rf ./*.vtk && rm -rf ./*.png')

    kwargs = {
        'lattice': lattice,
        'omega': omega,
        'nx': nx,
        'ny': ny,
        'nz': 0,
        'precision': precision,
        'io_rate': timestep_stride,
        'return_fpost': True    # Need to retain fpost-collision for computation of lift and drag
    }
    # characteristic time
    tc = prescribed_vel/diam
    niter_max = timestep_end
    if timestep_end < int(100//tc):
        print(colored("WARNING: timestep_end is too small, Karman flow may not appear", "red"))
    sim = Cylinder(**kwargs)
    sim.run(niter_max)
    return sim.saved_data

In [4]:
generated_data = generate_sim_dataaset(10, 0, 500, 20000)

**** Simulation Parameters for Cylinder ****
            Parameter | Value
--------------------------------------------------
                Omega | 1.971608832807571
     Grid Points in X | 220
     Grid Points in Y | 41
     Grid Points in Z | 0
       Dimensionality | 2
     Precision Policy | f64/f64
         Lattice Type | D2Q9
      Checkpoint Rate | 0
 Checkpoint Directory | ./checkpoints
  Downsampling Factor | 1
      Print Info Rate | 100
             I/O Rate | 500
        Compute MLUPS | False
   Restore Checkpoint | False
              Backend | gpu
    Number of Devices | 1
Time to create the grid mask: 0.10180521011352539
Time to create the local masks and normal arrays: 0.8365881443023682
         To set explicit initial density and velocity, use self.initialize_macroscopic_fields.


Saving data at timestep 20000/20000: 100%|██████████| 20001/20001 [00:04<00:00, 4482.62it/s]
